In [1]:
import pandas as pd
import os
import numpy as np
import sklearn.linear_model

In [5]:
os.getcwd()

'c:\\Users\\Ryan Wu\\VS Code Stuff\\NUANCE'

In [2]:
df = pd.read_csv("SHYNE Real Data.csv", parse_dates=True)

C:\Users\Ryan Wu\AppData\Local\Temp\ipykernel_25084\488733016.py:1: DtypeWarning: Columns (4,9,14,15,17,19,21,22,23,24,25,27,28,29,30,31,32,33,34,36,37,38,47,51,57,58,59,61,62,63,74,75,84,100) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("SHYNE Real Data.csv", parse_dates=True)


## Create Custom Dataframe for Visuals
- Filter for Instrument product type
- Only keeps hours, product name, start time, and end time columns
- Adds facility acronym tag in new column (Facilities not parsed for will have value of 0)
- Includes all types of usage (Reservation, Overage, etc.)

In [3]:
df_2 = df.loc[df["Product Type"] == "Instrument"]
df_2 = df_2[["Actual Hours", "Actual Start Time", "Actual End Time", "Product", "Year", "Facility"]]

facilities = ["EPIC", "Keck-II", "SPID", "BioCryo", "NUFAB", "NUFAB"]
instruments = [df_2["Product"].str.startswith("EPIC"), df_2["Product"].str.startswith("Keck-II"), 
               df_2["Product"].str.startswith("SPID"), df_2["Product"].str.startswith("BioCryo"), df_2["Facility"] == "NUFAB", df_2["Facility"].str.startswith("NU Micro")]


df_2["Facility_Acr"] = np.select(instruments, facilities)

#Creates new column with pandas datetime objects
df_2["datetime"] = pd.to_datetime(df_2["Actual Start Time"])

## Create Hour Interval Rows for Hour-By-Hour Usage
prob modify df for final to not waste memory instead of creating copy 
- code block below takes time but should be less than 2-3 minutes


In [111]:
#Parses string containing date for hour (left) and minute (right)
def left(series):
    if not pd.isna(series):
        return int(series[-5:-3])
def right(series):
    if not pd.isna(series):
        return int(series[-2:])
    
#Returns number of minutes the instrument was used in the current hour    
def time(row, curr):
    end_hr = row["end_hour"]
    if end_hr < row["start_hour"]:
        end_hr += 24
        if curr < row["start_hour"]:
            curr += 24
    if row["start_hour"] < curr < end_hr:
        return 60
    elif row["start_hour"] == end_hr and curr == end_hr:
        return row["end_min"] - row["start_min"]
    elif row["start_hour"] == curr:
        return 60 - row["start_min"]
    elif row["end_hour"] == curr:
        return row["end_min"]
    else:
        return 0

def get_hours(start, end, curr_hr):
    temp_df = pd.DataFrame()
    temp_df["start_hour"] = start.apply(left)
    temp_df["start_min"] = start.apply(right)
    temp_df["end_hour"] = end.apply(left)
    temp_df["end_min"] = end.apply(right)

    result = temp_df.apply(time, axis=1, curr=curr_hr)
    return result

for i in range(24):
    df_2[i] = get_hours(df_2["Actual Start Time"], df_2["Actual End Time"], i)

## Linear Regression Modeling
Creates two custom dataframes for month and day modeling



In [13]:
start = np.datetime64("2015", "M") #Used as the baseline for the date offset with a value of 0
day_start = np.datetime64("2021", "M")

#Gets time difference in months from current datetime to the start of 2015
def get_month_offset(datetime):
    if not pd.isna(datetime):
        return int((datetime - start) / np.timedelta64(1, "D") / 30)
    else:
        return 0

#Creates dataframe for month learning models
def get_hours_by_month(datetime):
    temp_df = pd.DataFrame()
    temp_df["MonthOffset"] = datetime.apply(get_month_offset)
    temp_df["Hours"] = df_2["Actual Hours"]
    return temp_df

model_df = get_hours_by_month(df_2["datetime"]).dropna(how="any")
model_df = model_df.groupby("MonthOffset").sum()
model_df["MonthOffset"] = model_df.index


In [14]:
#Gets time difference in days from current datetime to the start of 2015
def get_day_offset(datetime):
    if not pd.isna(datetime):
        if int((datetime - day_start) / np.timedelta64(1, "D")) > 0:
            return int((datetime - day_start) / np.timedelta64(1, "D"))
        else:
            return 0
    else:
        return 0
    
#Creates dataframe for day learning models
def get_hours_by_day(datetime):
    temp_df = pd.DataFrame()
    temp_df["DayOffset"] = datetime.apply(get_day_offset)
    temp_df["Hours"] = df_2["Actual Hours"]
    return temp_df

day_model_df = get_hours_by_day(df_2["datetime"]).dropna(how="any")
day_model_df = day_model_df.groupby("DayOffset").sum()
day_model_df["DayOffset"] = day_model_df.index

## Linear Modeling
- Creates two OLS linear models for month and day to overall usage with SKLearn
- MAQ Software uses MonthDate and DayDate versus Hour columns

In [8]:
model = sklearn.linear_model.LinearRegression()
x = model_df["MonthOffset"].to_frame()
y = model_df["Hours"].to_frame()

model.fit(x, y)

def get_predictions(month_offset):
    return int(month_offset * model.coef_ + model.intercept_)

model_df["OLS"] = model_df["MonthOffset"].apply(get_predictions)
model_df = model_df.iloc[1: , :]

def get_month_date(month_offset):
    return start + np.timedelta64(month_offset, "M")

model_df["MonthDate"] = model_df["MonthOffset"].apply(get_month_date)


C:\Users\Ryan Wu\AppData\Local\Temp\ipykernel_29644\1115468798.py:8: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return int(month_offset * model.coef_ + model.intercept_)


In [24]:
day_model = sklearn.linear_model.LinearRegression()
x = day_model_df["DayOffset"].to_frame()
y = day_model_df["Hours"].to_frame()

day_model.fit(x, y)

def get_predictions(day_offset):
    return int(day_offset * day_model.coef_ + day_model.intercept_)

day_model_df["OLS"] = day_model_df["DayOffset"].apply(get_predictions)
day_model_df = day_model_df.iloc[1: , :]

def get_day_date(day_offset):
    return day_start + np.timedelta64(day_offset, "D")

day_model_df["DayDate"] = day_model_df["DayOffset"].apply(get_day_date)

C:\Users\Ryan Wu\AppData\Local\Temp\ipykernel_29644\2076738162.py:8: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return int(day_offset * day_model.coef_ + day_model.intercept_)


In [1]:
#Dataset for Power BI visuals
df_2.to_csv('visuals.csv')

NameError: name 'df_2' is not defined

In [92]:
#Dataframe for month-by-month learning models
model_df.to_csv("month_model_data.csv")

In [26]:
#Dataframe for day-by-day learning models
day_model_df.to_csv("day_model_data.csv")